# Time Series Anomaly Detector

We will use a concept known as Bollinger Bands to discover when behavior changes from what is "expected" or normal. Bollinger Bands is a volatility indicator and commonly used in stock market trading. 

Scenario: Discover anomalies in number of web pages accessed in a day by a user. Is there a bot copying the curriculum? 

We will accomplish this by breaking down to the following tasks:

1. acquire the data
2. prepare the data
3. make the analysis process (which we will discuss later) work over all users. 
4. Turn the analysis process and calculations into a function that can be used to loop through for each user. 
5. Test the function on a single user. 
6. Analyze by looping over all users. 

The analysis process will look like this, for each user: 

1. compute necessary metrics to arrive at the final metric, %b (percent-b).
2. add user id to the dataframe that contains all the metrics, including %b. 
3. filter to rows where %b indicates anomaly (i.e. > 1)
4. append rows of anomalies of new user to previous users' anomalous activity. 

Finally, we will do a quick sample of exploration of the anomalies. There is much more you can do! 

Your exercise will be to add comments, markdown, and docstrings to the code in this lesson. 


- Google Slide Presentation:     https://docs.google.com/presentation/d/1hLFy6cWmJ4-bVUNykEGvXWxHnCwYMuFlsU0qwzxLCw4/edit?usp=sharing

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns


**Acquire**

After doing some research, some experimentation of performing actions and watching the logs, we discovered what each of the fields represent. We then parse and name the fields accordingly. 

In [2]:
colnames = ['date', 'endpoint', 'user_id', 'cohort_id', 'source_ip']
df = pd.read_csv("anonymized-curriculum-access-07-2021.txt", 
                 sep="\s", 
                 header=None, 
                 names = colnames, 
                 usecols=[0, 2, 3, 4, 5])
df.head()

,date,endpoint,user_id,cohort_id,source_ip
0,2018-01-26,/,1,8.0,97.105.19.61
1,2018-01-26,java-ii,1,8.0,97.105.19.61
2,2018-01-26,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,2018-01-26,slides/object_oriented_programming,1,8.0,97.105.19.61
4,2018-01-26,javascript-i/conditionals,2,22.0,97.105.19.61


**Prepare**


In [3]:
df.date = pd.to_datetime(df.date)
df = df.set_index(df.date)

In [ ]:
# resample daily counts for number of page views

pages = df['endpoint'].resample('d').count()
pages.head()

In [ ]:
#plot pages
pages.plot()

No need to split because we are not modeling, we are using statistics to identify low probability cases. 

## Exponential Moving Average

SMA time series are much less noisy than the time series of the original data points. 
The challenge with SMA, however, is that the values of SMA lag the original values. This means that changes in the trend are only seen with a delay (lag) of L time units. 

Exponential Moving Average (EMA) helps reduce the lag induced by the use of the SMA. It does this by putting more weight on more recent observations, while the SMA weights all observations equally.

The EMA function looks like this: 

$EMA_{t}= \alpha * (t_{0} - EMA_{t-1}) + EMA_{t-1}$

Where: 

- M = Number of time periods, span of the window

- $t_{0}$ = Latest value

- $t-1$ = Previous value

- $EMA_{t-1}$ = Exponential moving average of previous day. 

- The multiplier: $\alpha = \frac{2}{M+1}$

However, we will use the pandas ewm (Exponential Weighted functions) to compute our EMA. 
So we just need to define the following: 

- M = `span` argument = number of time periods. We will try 7 days, 14 days, and 30 days. 

- Notice how there are no missing values. ewm() will use as many values are available to compute the mean. So if the span is 7 days, but it is on the first day of data available, the EMA will equal the first value. 

In [ ]:
# 7 day EMA
ema_7d = 

In [ ]:
# 14 days EMA
ema_14d = 

In [ ]:
# 30 days EMA
ema_30d = 

In [ ]:
# 90 days EMA
ema_90d = 

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))

ax.plot(pages.index, pages, label='Daily', alpha=.5)

ax.plot(pages.index, ema_7d, label = '7-day EMA')
ax.plot(pages.index, ema_14d, label = '14-day EMA')
ax.plot(pages.index, ema_30d, label = '30-day EMA')
ax.plot(pages.index, ema_90d, label = '30-day EMA')

ax.legend(loc='best')
ax.set_ylabel('Number of pages')

plt.show()

## Bollinger Bands and %b

**Bollinger Bands**

- a volatility indicator and commonly used in stock market trading. 

- Made up of 3 lines, the Upper Band (UB), the Lower Band (LB) and the Midband.  

**Midband**

- The Exponential Moving Average

- `midband = train.ewm(span=30).mean()`

**Upper & Lower Band**

- UB/LB = Midband +/- stdev * K

- `stdev = train.ewm(span=30).std()` 

- K = the number of standard deviations to go up and down from the EMA

**%b, Percent Bandwidth**

- Shows where the last value sits in relation to the bands

- $\%b = \frac{last-LB}{UB-LB}$ 

- %b > 1 => point lies above UB

- %b < 0 => point lies below LB

- %b == .5 => point lies on the midband. 

**Bandwidth** 

- The width of the bands

- $Bandwidth = \frac{(UB-LB)}{Midband}$


In [ ]:
# set the window span
span = 30

# compute midband


In [ ]:
# compute exponential stdev


In [ ]:
# compute upper and lower bands


In [ ]:
# concat pages, midband, lb and ub to create a new df 'my_df'


Plot the bands

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

ax.plot(my_df.index, my_df.pages, label='Number of Pages')

ax.plot(my_df.index, my_df.midband, label = '30-day EMA/midband')
ax.plot(my_df.index, my_df.ub, label = 'Upper Band')
ax.plot(my_df.index, my_df.lb, label = 'Lower Band')

ax.legend(loc='best')
ax.set_ylabel('Number of pages')

plt.show()

Compute %b

$\%b = \frac{last-LB}{UB-LB}$ 



Compute pct_b for each point  

Now, where do we see anomalies? We will search for %b values > 1. We don't need to search for values < 0 because with this example, a low extreme is not something we are concerned about. 

## Pull it all together!

1. acquire
2. prep with user
3. compute features
    - compute midband
    - compute standard deviation
    - compute upper & lower band
    - create df with metrics
    - compute %b
    - add user_id to dataframe
4. Plot
5. search for anomalies

In [3]:
def acquire(file_name, column_names):
    return pd.read_csv(file_name, sep="\s", header=None, names=column_names, usecols=[0, 2, 3, 4, 5])

def prep(df, user):
    df = df[df.user_id == user]
    df.date = pd.to_datetime(df.date)
    df = df.set_index(df.date)
    pages = df['endpoint'].resample('d').count()
    return pages

def compute_pct_b(pages, span, weight, user):
    midband = pages.ewm(span=span).mean()
    stdev = pages.ewm(span=span).std()
    ub = midband + stdev * weight
    lb = midband - stdev * weight
    bb = pd.concat([ub, lb], axis=1)
    my_df = pd.concat([pages, midband, bb], axis=1)
    my_df.columns = ['pages', 'midband', 'ub', 'lb']
    my_df['pct_b'] = (my_df['pages'] - my_df['lb'])/(my_df['ub'] - my_df['lb'])
    my_df['user_id'] = user
    return my_df

def plt_bands(my_df, user):
    fig, ax = plt.subplots(figsize=(12,8))
    ax.plot(my_df.index, my_df.pages, label='Number of Pages, User: '+str(user))
    ax.plot(my_df.index, my_df.midband, label = 'EMA/midband')
    ax.plot(my_df.index, my_df.ub, label = 'Upper Band')
    ax.plot(my_df.index, my_df.lb, label = 'Lower Band')
    ax.legend(loc='best')
    ax.set_ylabel('Number of Pages')
    plt.show()
    
def find_anomalies(df, user, span, weight):
    pages = prep(df, user)
    my_df = compute_pct_b(pages, span, weight, user)
#     plt_bands(my_df, user)
    return my_df[my_df.pct_b > 1]

Acquire

In [4]:
file_name = "anonymized-curriculum-access-07-2021.txt"
column_names = ['date', 'endpoint', 'user_id', 'cohort_id', 'source_ip']
df = acquire(file_name, column_names)
df.head()

,date,endpoint,user_id,cohort_id,source_ip
0,2018-01-26,/,1,8.0,97.105.19.61
1,2018-01-26,java-ii,1,8.0,97.105.19.61
2,2018-01-26,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,2018-01-26,slides/object_oriented_programming,1,8.0,97.105.19.61
4,2018-01-26,javascript-i/conditionals,2,22.0,97.105.19.61


Test on a single user

In [5]:
user = 341
span = 30
weight = 3

anomalies = pd.DataFrame()
user_df = find_anomalies(df, user, span, weight)
anomalies = pd.concat([anomalies, user_df], axis=0)

In [6]:
anomalies

,pages,midband,ub,lb,pct_b,user_id
date,,,,,,
2019-03-03,272,24.721632,232.200343,-182.757078,1.095913,341
2020-02-18,3,0.193548,2.442380,-2.055283,1.123980,341
2020-03-10,4,0.305768,3.479509,-2.867973,1.082000,341
2020-04-21,109,7.250725,88.818626,-74.317176,1.123709,341
2020-07-13,52,6.283047,45.711487,-33.145393,1.079746,341
2020-10-03,11,0.991326,10.107234,-8.124582,1.048967,341
2020-10-25,10,0.873729,9.489350,-7.741892,1.029635,341
2021-02-09,10,0.659630,8.244463,-6.925203,1.115727,341


Loop through all users 

In [9]:
span = 30
weight = 3.5
anomalies = pd.DataFrame()
for u in list(df.user_id.unique()):
    user_df = find_anomalies(df, u, span, weight)
    anomalies = pd.concat([anomalies, user_df], axis=0)
    

In [10]:
anomalies

,pages,midband,ub,lb,pct_b,user_id
date,,,,,,
2019-01-29,44,3.623334,42.081344,-34.834677,1.024945,1
2019-07-08,101,8.248768,96.507472,-80.009936,1.025451,1
2018-10-15,9,0.581421,8.457414,-7.294571,1.034446,3
2019-01-09,4,0.262470,3.802115,-3.277175,1.027953,3
2019-04-06,2,0.129825,1.889712,-1.630061,1.031334,3
...,...,...,...,...,...,...
2021-06-02,72,10.456225,71.377304,-50.464854,1.005111,925
2021-06-29,32,3.881578,31.571606,-23.808449,1.007736,929
2021-06-11,44,5.582783,43.858655,-32.693089,1.001846,961


Explore anomalies

In [ ]:
# look at value counts for pages in anomalies


This is worth investigating! 

## Exercises

**file name:** time_series_anomaly_detection.py or time_series_anomaly_detection.ipynb

The dataset for these exercises lives in the Codeup Data Science MySQL Server. The database name is `curriculum_logs`.

Go through the lesson commenting code, adding docstrings, and adding markdown to support what is happening. 

Bonus:

Discover users who are accessing our curriculum pages way beyond the end of their codeup time. What would the dataframe look like? Use time series method for detecting anomalies, like exponential moving average with %b.

Can you label students who are viewing both the web dev and data science curriculum?
Can you label students by the program they are in? 
Can you label users by student vs. staff?